In [ ]:
!python --version

Python 3.11.13


In [ ]:
!pip install python-dotenv evaluate rouge_score
!pip install -U datasets==2.17.0
# !pip install transformers==4.41.2 peft==0.11.1 accelerate==0.30.1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a45d059b5b9b0b720e713dd616181bf56987037c8ca5e94c34c1c2bd34258d4f
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are in

In [ ]:
!pip list

Package                               Version
------------------------------------- -------------------
absl-py                               1.4.0
accelerate                            1.8.1
aiofiles                              24.1.0
aiohappyeyeballs                      2.6.1
aiohttp                               3.11.15
aiosignal                             1.4.0
alabaster                             1.0.0
albucore                              0.0.24
albumentations                        2.0.8
ale-py                                0.11.2
altair                                5.5.0
annotated-types                       0.7.0
antlr4-python3-runtime                4.9.3
anyio                                 4.9.0
argon2-cffi                           25.1.0
argon2-cffi-bindings                  21.2.0
array_record                          0.7.2
arviz                                 0.21.0
astropy                               7.1.0
astropy-iers-data                     0.2025.7.14.0.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset
from transformers import (
    GenerationConfig,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    TrainingArguments,
    Trainer,
)
from peft import get_peft_model, LoraConfig, TaskType, PeftModel, PeftConfig
import evaluate
import torch
import os
from dotenv import load_dotenv
from tqdm import tqdm
import json

In [ ]:
os.listdir("/content/drive/MyDrive/python_projects/fine-tuning-LLMs")

['.env', 'outputs', 'logs', 'models']

In [ ]:
env_path = "/content/drive/MyDrive/python_projects/fine-tuning-LLMs/.env"

# Load environment variables
load_dotenv(env_path)

True

In [ ]:
model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Handle padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
ds = load_dataset("knkarthick/samsum")

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/download/streaming_download_manager.py:784: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


Generating validation split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/download/streaming_download_manager.py:784: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/download/streaming_download_manager.py:784: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


In [ ]:
train_data, val_data, test_data = ds['train'], ds['validation'], ds['test']
ds = None

In [ ]:
train_data.shape, val_data.shape, test_data.shape

((14732, 3), (818, 3), (819, 3))

In [ ]:
train_data[0]

{'id': '13818513',
 'dialogue': "Amanda: I baked  cookies. Do you want some?\nJerry: Sure!\nAmanda: I'll bring you tomorrow :-)",
 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.'}

### see sample data

In [ ]:
sample_indices = [10, 40, 765]
for i, sample_index in enumerate(sample_indices):
  print('#'*50)
  print('-'*50)
  print(f'Example {i+1}')
  print('-'*50)
  print('dialogue:')
  print(test_data[sample_index]['dialogue'])
  print('-'*50)
  print('summary:')
  print(test_data[sample_index]['summary'])
  print('-'*50)
  print('#'*50)
  print()

##################################################
--------------------------------------------------
Example 1
--------------------------------------------------
dialogue:
Wanda: Let's make a party!
Gina: Why?
Wanda: beacuse. I want some fun!
Gina: ok, what do u need?
Wanda: 1st I need too make a list
Gina: noted and then?
Wanda: well, could u take yours father car and go do groceries with me?
Gina: don't know if he'll agree
Wanda: I know, but u can ask :)
Gina: I'll try but theres no promisess
Wanda: I know, u r the best!
Gina: When u wanna go
Wanda: Friday?
Gina: ok, I'll ask
--------------------------------------------------
summary:
Wanda wants to throw a party. She asks Gina to borrow her father's car and go do groceries together. They set the date for Friday. 
--------------------------------------------------
##################################################

##################################################
--------------------------------------------------
Example 2
-------

In [ ]:
text = 'How to Fine Tune LLMs?'

text_encoded = tokenizer(text, return_tensors='pt')

text_decoded = tokenizer.decode(text_encoded["input_ids"][0], skip_special_tokens=True)

print('ENCODED TEXT:')
print(text_encoded["input_ids"][0])
print('\nDECODED TEXT:')
print(text_decoded)

ENCODED TEXT:
tensor([  571,    12, 11456, 16008,    15,   301, 11160,     7,    58,     1])

DECODED TEXT:
How to Fine Tune LLMs?


In [ ]:
sample_indices = [10, 40, 765]
for i, sample_index in enumerate(sample_indices):
  dialogue = test_data[sample_index]['dialogue']
  summary = test_data[sample_index]['summary']
  prompt = f'''
summarize the below conversation

{dialogue}

summary:
  '''
  # Input constructed prompt instead of the dialogue.
  inputs = tokenizer(prompt, return_tensors='pt')
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens=50,
      )[0],
      skip_special_tokens=True
  )
  print('#'*50)
  print('-'*50)
  print(f'Example {i+1}')
  print('-'*50)
  print(f'prompt:\n{prompt}')
  print('-'*50)
  print(f'summary:\n{summary}')
  print('-'*50)
  print(f'LLM output:\n{output}')
  print('-'*50)
  print('#'*50)
  print()

##################################################
--------------------------------------------------
Example 1
--------------------------------------------------
prompt:

summarize the below conversation

Wanda: Let's make a party!
Gina: Why?
Wanda: beacuse. I want some fun!
Gina: ok, what do u need?
Wanda: 1st I need too make a list
Gina: noted and then?
Wanda: well, could u take yours father car and go do groceries with me?
Gina: don't know if he'll agree
Wanda: I know, but u can ask :)
Gina: I'll try but theres no promisess
Wanda: I know, u r the best!
Gina: When u wanna go
Wanda: Friday?
Gina: ok, I'll ask

summary:
  
--------------------------------------------------
summary:
Wanda wants to throw a party. She asks Gina to borrow her father's car and go do groceries together. They set the date for Friday. 
--------------------------------------------------
LLM output:
Wanda and Gina are going to make a party on Friday.
--------------------------------------------------
##########

In [ ]:
dialogues = test_data['dialogue']
len(dialogues)

819

In [ ]:
base_model_outputs = []

for datum in tqdm(test_data):
  dialogue = datum['dialogue']
  summary = datum['summary']
  prompt = f'''
summarize the below conversation

{dialogue}

summary:
  '''

  inputs = tokenizer(prompt, return_tensors='pt')
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens=50,
      )[0],
      skip_special_tokens=True
  )
  base_model_outputs.append(output)

100%|██████████| 819/819 [35:11<00:00,  2.58s/it]


In [ ]:
len(base_model_outputs)

819

In [ ]:
test_data_json = {
    'dialogue': test_data['dialogue'],
    'summary': test_data['summary'],
    'base_model': base_model_outputs
}

In [ ]:
type(test_data['dialogue'])

list

In [ ]:
with open('/content/drive/MyDrive/python_projects/fine-tuning-LLMs/outputs/test_data_json.json', 'w') as f:
  json.dump(test_data_json, f, indent=4)

In [ ]:
with open('/content/drive/MyDrive/python_projects/fine-tuning-LLMs/outputs/test_data_json.json', 'r') as f:
  test_data_json = json.load(f)

In [ ]:
# Load metrics
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

In [ ]:
rouge_result = rouge.compute(predictions=test_data_json['base_model'], references=test_data_json['summary'])
bleu_result = bleu.compute(predictions=test_data_json['base_model'], references=test_data_json['summary'])

In [ ]:
rouge_result, bleu_result

({'rouge1': np.float64(0.46970526766031273),
  'rouge2': np.float64(0.22877469275313655),
  'rougeL': np.float64(0.3928422518517964),
  'rougeLsum': np.float64(0.3926999461640388)},
 {'bleu': 0.16670307814224683,
  'precisions': [0.5256744428515574,
   0.2489846492737661,
   0.14808870732419024,
   0.09023198075878656],
  'brevity_penalty': 0.8151746242214558,
  'length_ratio': 0.8303213937885511,
  'translation_length': 15346,
  'reference_length': 18482})